In [ ]:
import os
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [ ]:
column_names = [
    "open_time",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "close_time",
    "quote_asset_volume",
    "number_of_trades",
    "taker_buy_base_asset_volume",
    "taker_buy_quote_asset_volume",
    "ignore"
]

In [ ]:
db_path = "binance_klines.db"
data_dir = "binance_klines"  # Folder where your CSVs are organized by symbol

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [ ]:
# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS klines (
    symbol TEXT,
    open_time INTEGER,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL,
    close_time INTEGER,
    quote_asset_volume REAL,
    number_of_trades INTEGER,
    taker_buy_base_asset_volume REAL,
    taker_buy_quote_asset_volume REAL
)
""")
conn.commit()


In [ ]:
for symbol in tqdm(os.listdir(data_dir)):
    symbol_dir = os.path.join(data_dir, symbol)
    if not os.path.isdir(symbol_dir):
        continue
    for fname in os.listdir(symbol_dir):
        if fname.endswith(".csv"):
            fpath = os.path.join(symbol_dir, fname)
            print(f"Importing {fpath}")
            df = pd.read_csv(fpath, names=column_names, header=None)
            df = df.drop(columns=["ignore"])
            df["symbol"] = symbol
            # Reorder columns to match table
            df = df[["symbol"] + column_names[:-1]]
            df.to_sql("klines", conn, if_exists="append", index=False)

conn.close()

In [ ]:
db_path = "binance_klines.db"
symbol = "BTCUSDT"
start_time = 1704067200000  # Example: 2024-01-01 00:00:00 UTC in ms
end_time = 1706745599999    # Example: 2024-01-31 23:59:59 UTC in ms



In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
SELECT * FROM klines
WHERE symbol = ?
  AND open_time BETWEEN ? AND ?
ORDER BY open_time
"""

cursor.execute(query, (symbol, start_time, end_time))
rows = cursor.fetchall()
df = pd.DataFrame(rows,columns = column_names)

conn.close()

In [ ]:
df